# Load price data 

In [1]:
import numpy as np
import pandas as pd
import finplot as fplt
import json

In [2]:
symbol = 'FUTURES BTC-USDT'
#df = pd.read_csv('Binance_BTCUSDT_1m_1596240000000-1598918400000.csv')
#df = pd.read_csv('Binance_BTCUSDT_1m_1567987200000-1598918400000.csv', nrows=5000)
df = pd.read_csv('Binance_BTCUSDT_1m_1567987200000-1598918400000.csv')
df = df.rename(columns={'Open_time':'time', 'Open':'open', 'Close':'close', 'High':'high', 'Low':'low', 'Volume':'volume'})

np.size(df['close'])
df

,time,open,high,low,close,volume,Close_time,Quote_asset_volume,Number_of_trades,Taker_buy_base_asset_volume,Taker_buy_quote_asset_volume,Ignore
0,1567987200000,10391.63,10391.63,10391.63,10391.63,0.000,1567987259999,0.000000e+00,0,0.000,0.000000e+00,0
1,1567987260000,10391.63,10391.63,10391.63,10391.63,0.000,1567987319999,0.000000e+00,0,0.000,0.000000e+00,0
2,1567987320000,10391.63,10391.63,10391.63,10391.63,0.000,1567987379999,0.000000e+00,0,0.000,0.000000e+00,0
3,1567987380000,10391.63,10391.63,10391.63,10391.63,0.000,1567987439999,0.000000e+00,0,0.000,0.000000e+00,0
4,1567987440000,10391.63,10391.63,10391.63,10391.63,0.000,1567987499999,0.000000e+00,0,0.000,0.000000e+00,0
...,...,...,...,...,...,...,...,...,...,...,...,...
515516,1598918160000,11666.84,11666.84,11658.77,11659.74,48.465,1598918219999,5.652471e+05,223,22.097,2.576819e+05,0
515517,1598918220000,11659.73,11660.04,11655.21,11655.22,51.387,1598918279999,5.990682e+05,245,11.315,1.318951e+05,0
515518,1598918280000,11655.22,11659.17,11654.00,11658.11,48.126,1598918339999,5.610001e+05,277,21.635,2.521977e+05,0
515519,1598918340000,11658.11,11660.53,11658.11,11658.11,78.462,1598918399999,9.147883e+05,245,21.574,2.515419e+05,0


# MA 200 + BB 200 strategy backtesting

* Plot MA 200 and BB 200 with (3 * standard deviation)
* When price breaks MA 200 enter two positions
    * Position 1 exits with a profit of 2%
    * Position 2 exits in next MA 200 cross

In [3]:
window = 200

# Calculate MA
ma_200 = df['close'].rolling(window).mean()
ma_20 = df['close'].rolling(20).mean()

# Calculate Bollinger Bands
bb_top_3delta = ma_200 + 3 * df['close'].rolling(window).std()
bb_bot_3delta = ma_200 - 3 * df['close'].rolling(window).std()

In [4]:
def crossovers(price, ma):
    result = np.zeros(np.size(price))
    for i,x in enumerate(ma):
        result[i] = None
        if(x is not None):
            if(price[i-1] < x and price[i] > x):
                result[i] = price[i]
            elif(price[i-1] > x and price[i] < x):
                result[i] = price[i]
    return result

cross = crossovers(df['close'].values, ma_200.values)

In [5]:
def remove_frequent_entries(cross, miss):
    m = 0
    cleaning = False
    result = np.zeros(np.size(cross))
    for i,x in enumerate(cross):
        result[i] = None
        if(x is not None):
            if(cleaning == True):
                result[i] = None
                m = m + 1
            if(m >= miss):
                m = 0
                cleaning = False
            if(cleaning == False and x > 0):
                result[i] = x
                cleaning = True
                #print('cleaning ' + str(i) + ' ' + str(result[i]))
            
    return result
                
cross_clean = remove_frequent_entries(cross, 30)

# Plot

In [6]:
try:
    # create two plots
    ax, ax2 = fplt.create_plot(symbol, rows=2)

    # plot candle sticks
    candles = df[['time','open','close','high','low']]
    fplt.candlestick_ochl(candles, ax=ax)

    # overlay volume on the top plot
    volumes = df[['time','open','close','volume']]
    fplt.volume_ocv(volumes, ax=ax.overlay())

    # Plot indicators
    fplt.plot(df['time'], ma_200, ax=ax, legend='ma_200')
    fplt.plot(df['time'], ma_20, ax=ax, legend='ma_20')
    fplt.plot(df['time'], cross_clean, ax=ax, legend='crossover', width=2, style='o', color='#0591fc')
    #fplt.plot(df['time'], cross_clean, ax=ax, legend='crossover_clean', width=6, style='^')
    #fplt.plot(df['time'], bb_bot_3delta, ax=ax, legend='bb_3d_bot')

    #fplt.plot(df['time'], df['close'].iloc[position_index], ax=ax, legend='position', style='^')
    #fplt.plot(crossing_dates, crossing_price, ax=ax2, legend='cross')

    # restore view (X-position and zoom)
    fplt.autoviewrestore()

    # we're done
    fplt.show()
except:
    pass